# Chembl validation pipeline

In [1]:
import os
if not os.getcwd().endswith('src'):
    os.chdir('../src')
from conf import cell_lines_chembl, TSR_OUT_DRUG
from validations.chembl.chembl_loader import load_raw_chembl

# Run MIthril on Bin Chen TCGA data

In [38]:
DISEASE = 'LIHC' # todo add dictionary of cell line  to disease for chembl data|

### create Mithril input from TCGA fodl cahnge data

In [40]:
from conf import DATA_DIR

DATA_DIR
tcga = pd.read_excel(DATA_DIR+'BinChen2017'+os.sep+'41467_2017_BFncomms16022_MOESM249_ESM.xlsx', sheet_name = DISEASE+'_sig.csv')
tcga


C:\Users\los4\miniconda3\envs\drug_repurposing\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj,symbol,GeneID,landmark
0,?|155060,?|155060,246.542053,63.387397,271.093615,4.276775,2.096523,2.322021e-11,4.919731e-10,?,155060,NaN
1,?|653553,?|653553,980.160561,308.154958,1070.241741,3.473064,1.796209,8.513384e-06,7.002229e-05,?,653553,NaN
2,AADAT|51166,AADAT|51166,539.906063,2260.466429,309.267944,0.136816,-2.869691,1.150033e-66,1.149267e-63,AADAT,51166,NaN
3,ABAT|18,ABAT|18,14667.875458,36469.512297,11745.404032,0.322061,-1.634595,1.161602e-36,3.214599e-34,ABAT,18,NaN
4,ABCA8|10351,ABCA8|10351,1142.503781,2728.212260,929.942323,0.340861,-1.552743,4.159067e-18,1.928178e-16,ABCA8,10351,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1187,ZNF695|57116,ZNF695|57116,18.119918,0.749038,20.448455,27.299608,4.770808,9.450933e-05,5.933800e-04,ZNF695,57116,NaN
1188,ZNF716|441234,ZNF716|441234,12.960449,0.068870,14.688543,213.279970,7.736605,1.291987e-05,1.016635e-04,ZNF716,441234,NaN
1189,ZNF883|169834,ZNF883|169834,41.344740,6.280339,46.045062,7.331620,2.874132,3.883273e-05,2.714820e-04,ZNF883,169834,NaN
1190,ZSWIM5|57643,ZSWIM5|57643,278.072279,76.284806,305.121538,3.999768,1.999916,2.617952e-10,4.834879e-09,ZSWIM5,57643,NaN


In [45]:
fc=tcga[['id','log2FoldChange']]
def split_id(id_line):
    return id_line.split('|')[1]
fc['id']=fc['id'].apply(lambda x : split_id(x))
fc.set_index('id', inplace=True)
fc

C:\Users\los4\AppData\Local\Temp\ipykernel_17252\2076856619.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc['id']=fc['id'].apply(lambda x : split_id(x))


,log2FoldChange
id,
155060,2.096523
653553,1.796209
51166,-2.869691
18,-1.634595
10351,-1.552743
...,...
57116,4.770808
441234,7.736605
169834,2.874132


In [47]:
#export as mitrhil input
from conf import MITH_IN_DISEASE
fc.to_csv(MITH_IN_DISEASE+DISEASE+'_signature_gene_id.mi', header=None, sep='\t')

In [51]:
from step2_disease_run_MITHrIL import run_mithril, MITH_OUT_DISEASE

In [52]:
run_mithril(DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE)

java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 30 -i G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\disease_signature\LIHC_signature_gene_id.mi -organism hsa_v2023_03 -o G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\disease_signature_2025\LIHC_mith3.output.txt -p G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\disease_signature_2025\LIHC_mith3.perturbations.txt -verbose


# Run Drug repurposing for HEPG2_24h

In [2]:
cell_line = 'HEPG2'
pert_time = '24h'

In [3]:
DRUG_DIR = TSR_OUT_DRUG+'LINCS'+os.sep+cell_line+'_'+pert_time+'_drug_wise'+os.sep

## Convert FC to Mith for given drug

In [4]:
from conf import MITH_IN_DRUG, map_name_to_id
MITH_IN_DRUG

'G:\\My Drive\\unict\\unict 2024-25\\netcos\\MITHrIL\\input\\drug_signature\\'

In [5]:
import os
import pandas as pd
import pyreadr

In [7]:
df_list=[]
for i, drug_file in enumerate(os.listdir(DRUG_DIR)):
    drug = drug_file.rstrip('.Rds')
    result = pyreadr.read_r(DRUG_DIR+drug_file)
    print(drug)
    df = result[None] 
    df.set_index('gene_id', inplace=True)
    df.rename(columns = {'DE_log2_FC':drug}, inplace = True)
    
    df_list.append(df[drug])

matrix=pd.concat(df_list, axis=1)
print(matrix.shape)
print(matrix.columns)
print(matrix.head(3))

isonicotinohydroxamic-aci
tozasertib
clomifene
tamoxifen
BRD-A08658758
avrainvillamide-analog-3
vinblastine
IKK-2-inhibitor-V
MW-SHH-54
NSC-632839
MW-SHH-97
BRD-K00640357
diethylstilbestrol
bucladesine
alpha-estradiol
avrainvillamide-analog-1
trichostatin-a
sulforaphane
SIB-1893
equol
estradiol
pentamidine
KI-8751
COL-3
BRD-K75627148
BRD-K67774729
avrainvillamide-analog-4
BRD-K52160314
tyrphostin-AG-879
BRD-K35832492
tert-butylhydroquinone
hydroxyl
HDAC1-selective
podophyllotoxin
staurosporine
AT-9283
simvastatin
BRD-K57064803
malonoben
BRD-K98948170
BRD-K83694683
BRD-K16956545
CGP-53353
KUC111109N
niacin
avrainvillamide-analog-6
A-23187
apicidin
KUC111774N
AT1-SHH-09
crizotinib
BRD-K55896799
BRD-K64106162
tyrphostin-A9
BRD-K61894884
atorvastatin
BRD-K11533227
barasertib-HQPA
estrone
FL-HDAC-025
doxorubicin
arctigenin
avicin-
fulvestrant
BRD-K99636700
raloxifene
benzohydroxamic-aci
MK-5108
spiramide
BRD-K22587354
erastin
sirolimu
Ro-90-7501
pifithrin-mu
BRD-K58547375
BRD-K21001652
ENMD

In [11]:
# Save MITHrIL gene id based input file:
mith_input_filename = 'LINCS_'+cell_line+'_'+pert_time+'.mi'
matrix.to_csv(MITH_IN_DRUG+mith_input_filename, sep='\t', index=True,  index_label=False)
print('MITHrIL drug input saved in',MITH_IN_DRUG,'filename:',LINCS_metanalysis_filename_id)

MITHrIL drug input saved in G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\drug_signature\ filename: LINCS_HEPG2_24h.mi


## <b>MITHrIL batch run</b>


In [22]:
import subprocess
from conf import MITH_APP, MITH_OUT_DRUG, MITH_IN_DRUG
from step6_drug_run_MITHrIL import run_mithril_batch

In [26]:
cell_line+'_'+pert_time

'HEPG2_24h'

In [21]:
mithril_output_dir = MITH_OUT_DRUG+cell_line+'_'+pert_time
run_mithril_batch(mith_input_filename, MITH_APP, MITH_IN_DRUG, mithril_output_dir, n_thread="2", printc=True)


java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril-batch -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 2 -organism hsa_v2023_03 -i G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\drug_signature\LINCS_HEPG2_24h.mi -o G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\drug_signature_2025\ -p -verbose
